In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-11-30 20:43:55--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   112MB/s    in 0.2s    

2023-11-30 20:43:55 (112 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [69]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
df_books

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber
...,...,...,...
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger
271375,0525447644,From One to One Hundred,Teri Sloat
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker
271377,0192126040,Republic (World's Classics),Plato


In [89]:
df_ratings = df_ratings[df_ratings['rating']!=0]
df_ratings.reset_index(inplace=True, drop=True)
df_ratings

,user,isbn,rating
0,276726,0155061224,5.0
1,276729,052165615X,3.0
2,276729,0521795028,6.0
3,276744,038550120X,7.0
4,276747,0060517794,9.0
...,...,...,...
383847,276704,0743211383,7.0
383848,276704,0806917695,5.0
383849,276704,1563526298,9.0
383850,276709,0515107662,10.0


In [76]:
df_ratings = df_ratings[df_ratings['isbn'].isin(df_books['isbn'])]
df_ratings.reset_index(inplace=True, drop=True)
df_ratings

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0
...,...,...,...
1031170,276704,0876044011,0.0
1031171,276704,1563526298,9.0
1031172,276706,0679447156,0.0
1031173,276709,0515107662,10.0


In [79]:
# add your code here - consider creating a new cell for each section of code
df_books[df_books['title']=="I'll Be Seeing You"]

,isbn,title,author
45,0671888587,I'll Be Seeing You,Mary Higgins Clark
74047,0553567187,I'll Be Seeing You,LURLENE MCDANIEL


In [80]:
df_ratings[df_ratings['isbn']=='0671888587']

,user,isbn,rating
1830,277478,0671888587,0.0
3157,278137,0671888587,0.0
8480,39,0671888587,7.0
9342,487,0671888587,6.0
28846,7904,0671888587,0.0
...,...,...,...
1002112,268032,0671888587,8.0
1013658,271195,0671888587,0.0
1015003,271284,0671888587,0.0
1021021,273979,0671888587,0.0


In [81]:
cont_user = df_ratings.user.value_counts()
cont_user

11676     11144
198711     6456
153662     5814
98391      5779
35859      5646
          ...  
116792        1
116798        1
116801        1
116810        1
276721        1
Name: user, Length: 92107, dtype: int64

In [14]:
cont_books = df_ratings.isbn.value_counts()
cont_books

0316666343     707
0971880107     581
0385504209     487
0312195516     383
0679781587     333
              ... 
0140441905       1
0886777267       1
0671697951       1
0553560956       1
05162443314      1
Name: isbn, Length: 185973, dtype: int64

In [49]:
# Obtener los valores que tienen al menos 200 y 100 instancias
# user200 = cont_user[cont_user >= 200].index
# bokks100 = cont_books[cont_books >= 100].index
# user= df_ratings[df_ratings['user'].isin(user200)].copy()
# bokks = df_ratings[df_ratings['isbn'].isin(bokks100)].copy()
# df_ratings = df_ratings[df_ratings['user'].isin(user200) & df_ratings['isbn'].isin(bokks100)].copy()
#df = df_ratings['rating']
#df

In [55]:
# df_rat =df_ratings['user'].isin(user200)
# df_ratings[df_rat]

,user,isbn,rating
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1484,277427,0140067477,0.0
1495,277427,0142001740,0.0
...,...,...,...
1146825,275970,0440226430,0.0
1146852,275970,044651652X,6.0
1146893,275970,0451524934,0.0
1147151,275970,0684874350,0.0


In [11]:
#min(bokks.isbn.value_counts())

In [12]:
#min(df.user.value_counts())

In [13]:
#min(df.isbn.value_counts())

In [82]:
df_books[df_books['title']=="Where the Heart Is (Oprah's Book Club (Paperback))"]

,isbn,title,author
706,0446672211,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts


In [83]:
df_ratings[df_ratings['isbn'] == '0140120831']

,user,isbn,rating


In [84]:
df_books[df_books['isbn'] == '0140120831']

,isbn,title,author


In [17]:
#df[df['isbn']=='0345351525']

In [18]:
#samples = df['rating'].to_list()
#samples.reshape(-1, 1)
#lista = np.array(samples).reshape(1, -1)
#len(samples)
#lista

In [19]:
# neigh = NearestNeighbors(n_neighbors=6, radius=0.4)
# neigh.fit(lista)

In [20]:
#print(neigh.kneighbors([[1., 1., 1.]]))

In [21]:
# isbn = df_books['isbn'][df_books['title']=="Where the Heart Is (Oprah's Book Club (Paperback))"].values
# isbn[0]

In [22]:
# book_rating = df[df['isbn']==isbn[0]]
# book_rating = book_rating['rating'].to_list()
# print(book_rating)
# lista = np.array(book_rating).reshape(-1, 1)
# #print(lista)
# neigh = NearestNeighbors(n_neighbors=5, radius=0.4)
# neigh.fit(lista)
# lista

In [23]:
#rng = neigh.radius_neighbors([[8.]])
#rng

In [24]:
#distances, neighbor_indices = neigh.kneighbors(lista)

In [25]:
# Obtener los libros recomendados basados en los vecinos más cercanos
# recommended_books = []
# listo = []
# for idx, distance in zip(neighbor_indices[0], distances[0]):
#         listo.append([df_books.loc[idx, 'title'], distance])

# recommended_books.append("Where the Heart Is (Oprah's Book Club (Paperback))")
# # for idx, distance in zip(neighbor_indices[0][1:], distances[0][1:]):
# #         listo.append((df_books.loc[idx, 'title'], distance))

# recommended_books = [recommended_books, listo]
# recommended_books


In [26]:
# book_rating = df[df['isbn']==isbn[0]]
# book_rating

In [85]:
df_ratings[df_ratings['rating']==0]

,user,isbn,rating
0,276725,034545104X,0.0
2,276727,0446520802,0.0
5,276733,2080674722,0.0
7,276746,0425115801,0.0
8,276746,0449006522,0.0
...,...,...,...
1031165,276704,059032120X,0.0
1031166,276704,0679752714,0.0
1031168,276704,080410526X,0.0
1031170,276704,0876044011,0.0


In [28]:
# def get_recommends(book = ""):
#     distances, indices = neigh.kneighbors(lista)

#     # Extract book titles and distances
#     recommended_books = [
#         df_books.loc[idx, 'title'] for idx in indices[0] if df_books.loc[idx, 'title'] != book
#     ]
#     distances = distances[0][1:]  # Exclude the distance to the book itself

#     # Combine titles and distances into a list of lists
#     result = [book, list(zip(recommended_books, distances))]

#     return result
#df_ratings[df_ratings['title']==np.nan]

In [91]:
# isbn = df_books['isbn'][df_books['title']=="Where the Heart Is (Oprah's Book Club (Paperback))"].values
# isbn[0]
# book_rating = df[df['isbn']==isbn[0]]
# book_rating = book_rating['rating']
# #book_rating
# lista = np.array(book_rating).reshape(-1, 1)
# lista
# books_rating = df_ratings['rating'].values
# books_rating = np.array(books_rating).reshape(-1, 1)
# len(books_rating)
df_promedios = df_ratings.groupby('isbn')['rating'].mean().reset_index()
df_promedios.columns = ['isbn', 'rating']
#df_promedios[df_promedios['rating'].duplicated()]
#df_promedios[df_promedios['isbn']=='Ô½crosoft']
#df_promedios['rating'].reset_index()
df_promedios[df_promedios['isbn']=='0140001409']

,isbn,rating
8416,0140001409,8.142858


In [92]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommended_books = [book]
  #print(book)
  listo = []
  isbn = df_books['isbn'][df_books['title']==book].values
  book_rating = df_promedios[df_promedios['isbn']==isbn[0]]
  book_rating = book_rating['rating'].values
  lista = np.array(book_rating).reshape(-1, 1)
  suma_elementos = sum(lista)
  promedio = suma_elementos / len(lista)
  print(promedio[0])
  #todas las notas
  books_rating = df_promedios['rating'].values
  books_rating = np.array(books_rating).reshape(-1, 1)
  correspondencias = dict(zip(df_books['isbn'], df_books['title']))
  df_promedios['title'] = df_promedios['isbn'].map(correspondencias)
  print('tamaño de los datos, : ', len(df_promedios))
  neigh = NearestNeighbors(n_neighbors=5, radius=1)
  neigh.fit(books_rating)
  A = neigh.kneighbors_graph([promedio])
  A.toarray()
  #plt.plot(A)
  #print(A)
  distances, neighbor_indices = neigh.kneighbors([book_rating], 5)
  print(neighbor_indices)
  print(distances)
  for idx, distance in zip(neighbor_indices[0][1:], distances[0][1:]):
        listo.append([df_promedios.loc[idx, 'title'], distance])
  recommended_books = [recommended_books, listo]
  #neigh.kneighbors([[1., 1., 1.]])
  return recommended_books

In [93]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

8.142373
tamaño de los datos, :  149842
[[51802  5354 11337  4305  3377]]
[[0.         0.00048447 0.00048447 0.00048447 0.00048447]]
[["Where the Heart Is (Oprah's Book Club (Paperback))"], [['Ruins (The X-Files)', 0.000484466552734375], ['The Epic of Gilgamesh: An English Version With an Introduction (Penguin Classics)', 0.000484466552734375], ['Papillon', 0.000484466552734375], ['Neverwhere : A Novel', 0.000484466552734375]]]
8.142373
tamaño de los datos, :  149842
[[51802  5354 11337  4305  3377]]
[[0.         0.00048447 0.00048447 0.00048447 0.00048447]]
You haven't passed yet. Keep trying!


In [35]:
df_promedios['rating'][df_promedios['title']=="Where the Heart Is (Oprah's Book Club (Paperback))"].values

array([8.142373], dtype=float32)

In [36]:
df_ratings['rating'].unique()

array([ 5.,  3.,  6.,  8.,  7., 10.,  9.,  4.,  1.,  2.], dtype=float32)

In [36]:
df_promedios[df_promedios['title']=="I'll Be Seeing You"]

,isbn,rating,title


In [45]:
df_books[df_books['title']=="I'll Be Seeing You"]

,isbn,title,author
45,0671888587,I'll Be Seeing You,Mary Higgins Clark
74047,0553567187,I'll Be Seeing You,LURLENE MCDANIEL


In [44]:
df_ratings[df_ratings['isbn']=="0671888587"]

,user,isbn,rating
2158,277478,0671888587,0.0
3644,278137,0671888587,0.0
9617,39,0671888587,7.0
10729,487,0671888587,6.0
33292,7904,0671888587,0.0
...,...,...,...
1117151,268032,0671888587,8.0
1129862,271195,0671888587,0.0
1131229,271284,0671888587,0.0
1137934,273979,0671888587,0.0


In [30]:
df_promedios[df_promedios['title']=="The Weight of Water"]

,isbn,rating,title


In [75]:
df_promedios[''].loc[12032]

isbn      0140120831
rating      8.142858
title            NaN
Name: 12032, dtype: object

In [74]:
df_books[df_books['isbn']=='0140001409']

,isbn,title,author
9007,0140001409,Cold Comfort Farm,Stella Gibbons


In [5]:
df_books[df_books['isbn']=='0140120831']

,isbn,title,author
